# Extract-Transform-Load and Exploratory Data Analysis
This notebook contains all code for the prelimiatory analysis of the KDD Cup 98 datasets

## Loading the tidy datasets
The class TidyDataset holds all transforming steps to create a valid dataset for later use. It transforms:
    - boolean fields that are coded in various ways in the original dataset -> 0/1
    - date strings yymm -> individual columns with yy and mm, name prefixed with original field name
    - Multi-value (bytewise) categories -> individual categorical fields
    - ZIP codes are stripped of trailing dashes and a category created

In [3]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set working directory to main code folder

In [4]:
import os
os.chdir("../")

Setting the magic autoreload ensures modules are reloaded before execution.

In [5]:
%autoreload 2
import eda.tidy_dataset as tds

Loading learning and validation datasets

In [46]:
lrn = tds.TidyDataset("cup98LRN.txt")#, pull_stored=False) # To force reprocessing
learning = lrn.get_tidy_data()

In [31]:
learning_raw = lrn.get_raw_data()

In [ ]:
val = tds.TidyDataset("cup98VAL.txt")
validation = val.get_tidy_data()

## A first look at the learning datasets

Check that the learning and validation sets are really disjoint:

In [ ]:
set(learning.index.values) & set(validation.index.values) # if empty, they are disjoint

Some basic info on the dimensions and data types contained. Category- and object-features will have to be dealt with further.

In [19]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95412 entries, 95515 to 185114
Columns: 554 entries, OSOURCE to RFA_24_Amount
dtypes: bool(30), category(90), float64(74), int64(303), object(57)
memory usage: 328.4+ MB


The boolean columns:

In [14]:
bool_cols = learning.select_dtypes(include="bool")
bool_cols_dict = ["MAILCODE", "NOEXCH","RECINHSE","RECP3","RECPGVG","AGEFLAG","HOMEOWNR","MAJOR","COLLECT1","VETERANS","BIBLE","CATLG","HOMEE","PETS","CDPLAY","STEREO","PCOWNERS","PHOTO","CRAFTS","FISHER","GARDENIN","BOATS","WALKER","KIDSTUFF","CARDS","PLATES","PEPSTRFL","TARGET_B","HPHONE_D"]
list(set(bool_cols_dict)-set(bool_cols)) # Check we captured all. Must be empty

[]

Investigate result of boolean conversion.

1. Are all features now valid booleans? 
2. Are the correct values set?

In [43]:
for col in bool_cols_dict:
    print(col+": "+str(learning.loc[:,col].dtype)+".\t\t\tValues: "+str(set(learning.loc[:,col])))

for col in bool_cols_dict:
    print(+learning[col].value_counts())
    print(learning_raw[col].value_counts())
    print("-------------------------------")

MAILCODE: bool.			Values: {False, True}
NOEXCH: bool.			Values: {False, True}
RECINHSE: bool.			Values: {False, True}
RECP3: bool.			Values: {False, True}
RECPGVG: bool.			Values: {False, True}
AGEFLAG: bool.			Values: {False, True}
HOMEOWNR: bool.			Values: {False, True}
MAJOR: bool.			Values: {False, True}
COLLECT1: bool.			Values: {False, True}
VETERANS: bool.			Values: {False, True}
BIBLE: bool.			Values: {False, True}
CATLG: bool.			Values: {False, True}
HOMEE: bool.			Values: {False, True}
PETS: bool.			Values: {False, True}
CDPLAY: bool.			Values: {False, True}
STEREO: bool.			Values: {False, True}
PCOWNERS: bool.			Values: {False, True}
PHOTO: bool.			Values: {False, True}
CRAFTS: bool.			Values: {False, True}
FISHER: bool.			Values: {False, True}
GARDENIN: bool.			Values: {False, True}
BOATS: bool.			Values: {False, True}
WALKER: bool.			Values: {False, True}
KIDSTUFF: bool.			Values: {False, True}
CARDS: bool.			Values: {False, True}
PLATES: bool.			Values: {False, True}
PEPS

In [18]:
bool_cols.describe()

,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,AGEFLAG,HOMEOWNR,MAJOR,COLLECT1,...,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,PEPSTRFL,TARGET_B,HPHONE_D
count,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,...,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,True
freq,94013,95377,88709,93395,95298,93795,57344,52354,95118,90210,...,88282,82010,93384,84911,93876,94371,94852,50143,90569,47765


Let's have a look at the object columns. Object is anything that is not a simple numeric, boolean or character feature and all features that still contain mixed data.

In [44]:
object_cols = learning.select_dtypes(include="object")

In [45]:
object_cols.columns

Index(['OSOURCE', 'RFA_2R', 'RFA_2A', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A',
       'GEOCODE2', 'ODATEDW_year', 'ODATEDW_month', 'DOB_year', 'DOB_month',
       'ADATE_2_year', 'ADATE_2_month', 'ADATE_3_year', 'ADATE_3_month',
       'ADATE_4_year', 'ADATE_4_month', 'ADATE_5_year', 'ADATE_5_month',
       'ADATE_6_year', 'ADATE_6_month', 'ADATE_7_year', 'ADATE_7_month',
       'ADATE_8_year', 'ADATE_8_month', 'ADATE_9_year', 'ADATE_9_month',
       'ADATE_10_year', 'ADATE_10_month', 'ADATE_11_year', 'ADATE_11_month',
       'ADATE_12_year', 'ADATE_12_month', 'ADATE_13_year', 'ADATE_13_month',
       'ADATE_14_year', 'ADATE_14_month', 'ADATE_15_year', 'ADATE_15_month',
       'ADATE_16_year', 'ADATE_16_month', 'ADATE_17_year', 'ADATE_17_month',
       'ADATE_18_year', 'ADATE_18_month', 'ADATE_19_year', 'ADATE_19_month',
       'ADATE_20_year', 'ADATE_20_month', 'ADATE_21_year', 'ADATE_21_month',
       'ADATE_22_year', 'ADATE_22_month', 'ADATE_23_year', 'ADATE_23_month',
       'ADATE_24_y

In [37]:
learning.NOEXCH.value_counts()

False    95377
True        35
Name: NOEXCH, dtype: int64

In [30]:
set(object_cols.OSOURCE)

{' ',
 'AAA',
 'AAD',
 'AAM',
 'ABC',
 'ABD',
 'ABP',
 'ABW',
 'ACB',
 'ACC',
 'ACD',
 'ACG',
 'ACL',
 'ACN',
 'ACP',
 'ACR',
 'ACS',
 'ACT',
 'ADA',
 'ADD',
 'ADM',
 'ADR',
 'AEW',
 'AFP',
 'AFR',
 'AGF',
 'AGR',
 'AGS',
 'AHA',
 'AHB',
 'AHC',
 'AHL',
 'AHM',
 'AHN',
 'AHR',
 'AHS',
 'AI2',
 'AIC',
 'AIR',
 'AKF',
 'AKM',
 'ALD',
 'ALF',
 'ALL',
 'ALS',
 'ALZ',
 'AMB',
 'AME',
 'AMG',
 'AMH',
 'AMJ',
 'AML',
 'AMR',
 'AMS',
 'AMT',
 'AMU',
 'AMV',
 'AMZ',
 'ANN',
 'ANR',
 'ANT',
 'AP1',
 'APL',
 'APP',
 'AQK',
 'AQU',
 'AR1',
 'ARB',
 'ARC',
 'ARE',
 'ARF',
 'ARG',
 'ARM',
 'ARZ',
 'ASB',
 'ASC',
 'ASD',
 'ASE',
 'ASH',
 'ASK',
 'ASP',
 'ASS',
 'ATL',
 'ATP',
 'ATS',
 'AUD',
 'AUS',
 'AVD',
 'AVN',
 'AVS',
 'AWK',
 'BAN',
 'BAR',
 'BAS',
 'BBK',
 'BBO',
 'BBT',
 'BBU',
 'BBW',
 'BCN',
 'BDG',
 'BEA',
 'BED',
 'BEE',
 'BEL',
 'BET',
 'BFD',
 'BHB',
 'BHG',
 'BHL',
 'BHW',
 'BIB',
 'BIR',
 'BKS',
 'BLI',
 'BLT',
 'BLU',
 'BLV',
 'BMG',
 'BML',
 'BMS',
 'BOA',
 'BOS',
 'BRC',
 'BRE',
 '

Is this a processing problem? Check raw data!

In [ ]:
raw = lrn.get_raw_data()

In [ ]:
raw.info()

In [ ]:
raw[empty_cols]

In [ ]:
set(raw["RECPGVG"])

In [ ]:
set(learning["NOEXCH"])

In [ ]:
set(object_cols.HOMEOWNR)

The title code is a categorical field. It may be helpful to transform the code to the actual titles. However, as is seen below, several of the codes present in the data are not in the dictionary, leaving only the option to set these to empty, which would result in a loss of information. The numerical levels are therefore kept.

In [ ]:
learning.info()

In [ ]:
tcode_categories = {
        0: "_",
        1: "MR.",
        1001: "MESSRS.",
        1002: "MR. & MRS.",
        2: "MRS.",
        2002: "MESDAMES",
        3: "MISS",
        3003: "MISSES",
        4: "DR.",
        4002: "DR. & MRS.",
        4004: "DOCTORS",
        5: "MADAME",
        6: "SERGEANT",
        9: "RABBI",
        10: "PROFESSOR",
        10002: "PROFESSOR & MRS.",
        10010: "PROFESSORS",
        11: "ADMIRAL",
        11002: "ADMIRAL & MRS.",
        12: "GENERAL",
        12002: "GENERAL & MRS.",
        13: "COLONEL",
        13002: "COLONEL & MRS.",
        14: "CAPTAIN",
        14002: "CAPTAIN & MRS.",
        15: "COMMANDER",
        15002: "COMMANDER & MRS.",
        16: "DEAN",
        17: "JUDGE",
        17002: "JUDGE & MRS.",
        18: "MAJOR",
        18002: "MAJOR & MRS.",
        19: "SENATOR",
        20: "GOVERNOR",
        21002: "SERGEANT & MRS.",
        22002: "COLNEL & MRS.",
        24: "LIEUTENANT",
        26: "MONSIGNOR",
        27: "REVEREND",
        28: "MS.",
        28028: "MSS.",
        29: "BISHOP",
        31: "AMBASSADOR",
        31002: "AMBASSADOR & MRS.",
        33: "CANTOR",
        36: "BROTHER",
        37: "SIR",
        38: "COMMODORE",
        40: "FATHER",
        42: "SISTER",
        43: "PRESIDENT",
        44: "MASTER",
        46: "MOTHER",
        47: "CHAPLAIN",
        48: "CORPORAL",
        50: "ELDER",
        56: "MAYOR",
        59002: "LIEUTENANT & MRS.",
        62: "LORD",
        63: "CARDINAL",
        64: "FRIEND",
        65: "FRIENDS",
        68: "ARCHDEACON",
        69: "CANON",
        70: "BISHOP",
        72002: "REVEREND & MRS.",
        73: "PASTOR",
        75: "ARCHBISHOP",
        85: "SPECIALIST",
        87: "PRIVATE",
        89: "SEAMAN",
        90: "AIRMAN",
        91: "JUSTICE",
        92: "MR. JUSTICE",
        100: "M.",
        103: "MLLE.",
        104: "CHANCELLOR",
        106: "REPRESENTATIVE",
        107: "SECRETARY",
        108: "LT. GOVERNOR",
        109: "LIC.",
        111: "SA.",
        114: "DA.",
        116: "SR.",
        117: "SRA.",
        118: "SRTA.",
        120: "YOUR MAJESTY",
        122: "HIS HIGHNESS",
        123: "HER HIGHNESS",
        124: "COUNT",
        125: "LADY",
        126: "PRINCE",
        127: "PRINCESS",
        128: "CHIEF",
        129: "BARON",
        130: "SHEIK",
        131: "PRINCE AND PRINCESS",
        132: "YOUR IMPERIAL MAJEST",
        135: "M. ET MME.",
        210: "PROF."}

In [ ]:
new_cats = {str(k):str(v) for k,v in tcode_categories.items()}
new_cats
def set_new_tcode(old):
    if old in new_cats:
        return new_cats[old]
    else:
        return new_cats['0']
    
temp = learning.TCODE.cat.rename_categories(new_categories=new_cats)
temp.cat.categories

## US census data

In [ ]:
us_census = ["POP901", "POP902", "POP903", "POP90C1", "POP90C2", "POP90C3", "POP90C4", "POP90C5", "ETH1", "ETH2", "ETH3", "ETH4", "ETH5", "ETH6", "ETH7", "ETH8", "ETH9", "ETH10", "ETH11", "ETH12", "ETH13", "ETH14", "ETH15", "ETH16", "AGE901", "AGE902", "AGE903", "AGE904", "AGE905", "AGE906", "AGE907", "CHIL1", "CHIL2", "CHIL3", "AGEC1", "AGEC2", "AGEC3", "AGEC4", "AGEC5", "AGEC6", "AGEC7", "CHILC1", "CHILC2", "CHILC3", "CHILC4", "CHILC5", "HHAGE1", "HHAGE2", "HHAGE3", "HHN1", "HHN2", "HHN3", "HHN4", "HHN5", "HHN6", "MARR1", "MARR2", "MARR3", "MARR4", "HHP1", "HHP2", "DW1", "DW2", "DW3", "DW4", "DW5", "DW6", "DW7", "DW8", "DW9", "HV1", "HV2", "HV3", "HV4", "HU1", "HU2", "HU3", "HU4", "HU5", "HHD1", "HHD2", "HHD3", "HHD4", "HHD5", "HHD6", "HHD7", "HHD8", "HHD9", "HHD10", "HHD11", "HHD12", "ETHC1", "ETHC2", "ETHC3", "ETHC4", "ETHC5", "ETHC6", "HVP1", "HVP2", "HVP3", "HVP4", "HVP5", "HVP6", "HUR1", "HUR2", "RHP1", "RHP2", "RHP3", "RHP4", "HUPA1", "HUPA2", "HUPA3", "HUPA4", "HUPA5", "HUPA6", "HUPA7", "RP1", "RP2", "RP3", "RP4", "MSA", "ADI", "DMA", "IC1", "IC2", "IC3", "IC4", "IC5", "IC6", "IC7", "IC8", "IC9", "IC10", "IC11", "IC12", "IC13", "IC14", "IC15", "IC16", "IC17", "IC18", "IC19", "IC20", "IC21", "IC22", "IC23", "HHAS1", "HHAS2", "HHAS3", "HHAS4", "MC1", "MC2", "MC3", "TPE1", "TPE2", "TPE3", "TPE4", "TPE5", "TPE6", "TPE7", "TPE8", "TPE9", "PEC1", "PEC2", "TPE10", "TPE11", "TPE12", "TPE13", "LFC1", "LFC2", "LFC3", "LFC4", "LFC5", "LFC6", "LFC7", "LFC8", "LFC9", "LFC10", "OCC1", "OCC2", "OCC3", "OCC4", "OCC5", "OCC6", "OCC7", "OCC8", "OCC9", "OCC10", "OCC11", "OCC12", "OCC13", "EIC1", "EIC2", "EIC3", "EIC4", "EIC5", "EIC6", "EIC7", "EIC8", "EIC9", "EIC10", "EIC11", "EIC12", "EIC13", "EIC14", "EIC15", "EIC16", "OEDC1", "OEDC2", "OEDC3", "OEDC4", "OEDC5", "OEDC6", "OEDC7", "EC1", "EC2", "EC3", "EC4", "EC5", "EC6", "EC7", "EC8", "SEC1", "SEC2", "SEC3", "SEC4", "SEC5", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6", "VC1", "VC2", "VC3", "VC4", "ANC1", "ANC2", "ANC3", "ANC4", "ANC5", "ANC6", "ANC7", "ANC8", "ANC9", "ANC10", "ANC11", "ANC12", "ANC13", "ANC14", "ANC15", "POBC1", "POBC2", "LSC1", "LSC2", "LSC3", "LSC4", "VOC1", "VOC2", "VOC3", "HC1", "HC2", "HC3", "HC4", "HC5", "HC6", "HC7", "HC8", "HC9", "HC10", "HC11", "HC12", "HC13", "HC14", "HC15", "HC16", "HC17", "HC18", "HC19", "HC20", "HC21", "MHUC1", "MHUC2", "AC1", "AC2"]
len(us_census)

# Feature Extraction
All explanatory fields have to be numerical for the subsequent operations with scikit-learn. Here, the necessary feature extractions are performed.

See [scikit-learn: feature extraction](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
learning.dtypes

# Feature Selection
Meant to reduce dimensionality by selecting only features that are 'interesting enough' to be considered in order to boost performance of calculations / improve accuracy of the estimator
- By variance threshold
- Recursive Feature Elimination by Cross-Validation
- L1-based feature selection (Logistic Regression, Lasso, SVM)
- Tree-based feature selection

See [scikit-learn: feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)


# PCA

A first look at important features

In [ ]:
from sklearn import decomposition

In [ ]:
X = learning.drop(["TARGET_B","TARGET_D"],axis=1)

In [ ]:
n_comp = 3
pca = decomposition.PCA(n_components = n_comp)
pca.fit(X)
result = pd.DataFrame(pca.transform(X), columns=["PCA%i" % i for i in range(n_comp)], index=X.index)

In [ ]:
import cProfile
domain_spreader = tds.SymbolicFieldToDummies(learning,"RFA_24",["Recency", "Frequency", "Amount"])
cProfile.run('domain_spreader.spread()', sort='time')

In [ ]:
learning.head()

In [ ]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

In [ ]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt")

In [ ]:
raw = tidy.get_raw_data()

In [ ]:
spreader = tds.SymbolicFieldToDummies(
    raw, "RFA_24", ["Recency", "Frequency", "Amount"])
spreader.spread()